In [1]:
import os
import json

with open('./source_test_result/source_logical_error_rate.json', 'r') as f:
    source_data = json.load(f)

with open('./output_test_result/output_logical_error_rate.json', 'r') as f1:
    output_data = json.load(f1)

with open('./gold_test_result/gold_logical_error_rate.json', 'r') as f2:
    gold_data = json.load(f2)

In [2]:
import os
import pandas as pd

gold_file_path = '../logicalErrorFix_CodeT5_Linenumber/model/cpp/checkpoint-best-bleu-CodeT5-Line-Number/test_with_line_1.gold'
output_file_path = '../logicalErrorFix_CodeT5_Linenumber/model/cpp/checkpoint-best-bleu-CodeT5-Line-Number/test_with_line_1.output'

columns = ['index', 'code']

pd_gold = pd.read_csv(gold_file_path, sep = '\t', names=columns)
pd_output = pd.read_csv(output_file_path, sep = '\t', names=columns)

code_len = len(pd_gold)
exact_match_index = []

for index in range(code_len):
    if pd_gold.loc[index]["code"] == pd_output.loc[index]["code"]:
        exact_match_index.append(index)

In [3]:
import os
path = './CodeT5_LineNumber_cpp_gold/'
file_list = os.listdir(path)
test_result_dictionary = {}
data_type = ['gold', 'source', 'output']

for cpp in file_list:
    problem_id = cpp.split('_')[1]
    index_id = cpp.split('_')[3].split('.')[0]
    test_result_dictionary[problem_id + '_' + index_id] = []
    for dt in data_type:
        file_key = 'Problem_' + problem_id + '_' + dt + '_' + index_id

        try:
            if dt == 'gold':
                accuarcy = gold_data[file_key][0]
            elif dt == 'source':
                accuarcy = source_data[file_key][0]
            else:
                accuarcy = output_data[file_key][0]
        except KeyError:
            accuarcy = 'CE'
        test_result_dictionary[problem_id + '_' + index_id].append(accuarcy)

print(test_result_dictionary)
print(f'Total Data = {len(test_result_dictionary)}')

{'0_0': [0.0, 0.17000000000000004, 0.20999999999999996], '0_1': [0.0, 0.010000000000000009, 0.0], '0_2': [0.0, 1.0, 'CE'], '0_3': [0.0, 0.0, 'CE'], '100_524': [0.0, 0.28500000000000003, 1.0], '100_525': [0.0, 1.0, 1.0], '100_526': ['CE', 1.0, 1.0], '101_527': ['CE', 1.0, 1.0], '101_528': [0.0, 0.48, 0.48], '102_529': [0.995, 0.975, 1.0], '102_530': [0.0, 0.44999999999999996, 0.0], '102_531': [0.995, 0.995, 1.0], '103_532': [0.0, 0.84, 0.84], '103_533': [0.0, 0.84, 0.84], '104_534': [1.0, 1.0, 1.0], '104_535': [1.0, 1.0, 1.0], '104_536': ['CE', 1.0, 'CE'], '105_537': [0.995, 0.995, 0.16500000000000004], '105_538': [0.0, 1.0, 1.0], '105_539': [0.0, 0.050000000000000044, 0.050000000000000044], '105_540': [0.0, 0.83, 0.83], '105_541': [0.995, 0.995, 'CE'], '105_542': [0.0, 0.0, 0.0], '106_543': [0.0, 1.0, 0.0], '106_544': [0.0, 1.0, 1.0], '106_545': [0.0, 1.0, 1.0], '106_546': [0.0, 1.0, 'CE'], '106_547': [0.0, 1.0, 1.0], '106_548': [0.0, 1.0, 0.0], '106_549': [0.0, 1.0, 1.0], '106_550': [

In [4]:
clean_source = []
corrupted_source = []
minor_issues_source = []
compile_error_source = []

for key, index in test_result_dictionary.items():

    if index[1] == 0.0:
        clean_source.append(key)
    elif index[1] == 1.0:
        corrupted_source.append(key)
    elif index[1] == 'CE':
        compile_error_source.append(key)
    else:
        minor_issues_source.append(key)

print(f'Clean Source = {len(clean_source)}')
print(f'Corrupted Source = {len(corrupted_source)}')
print(f'Minor Issues Souce = {len(minor_issues_source)}')
print(f'Compile Error Source = {len(compile_error_source) - 7}')
print(f'Test Case Error 7')

Clean Source = 210
Corrupted Source = 110
Minor Issues Souce = 528
Compile Error Source = 6
Test Case Error 7


In [5]:
clean_output = []
corrupted_output = []
minor_issues_output = []
compile_error_output = []

for key, index in test_result_dictionary.items():

    if index[2] == 0.0:
        clean_output.append(key)
    elif index[2] == 1.0:
        corrupted_output.append(key)
    elif index[2] == 'CE':
        compile_error_output.append(key)
    else:
        minor_issues_output.append(key)

print(f'Clean output = {len(clean_output)}')
print(f'Corrupted output = {len(corrupted_output)}')
print(f'Minor Issues output = {len(minor_issues_output)}')
print(f'Compile Error output = {len(compile_error_output) - 7}')
print(f'Test Case Error 7')

Clean output = 218
Corrupted output = 145
Minor Issues output = 431
Compile Error output = 60
Test Case Error 7


In [6]:
clean_detect = len(clean_source)
clean_source_compile_error_output = 0
for key in clean_source:
    if test_result_dictionary[key][2] == 'CE':
        # Output Compile Error
        clean_detect -= 1
        clean_source_compile_error_output += 1
    elif test_result_dictionary[key][1] < test_result_dictionary[key][2]:
        # Source Error(0.0) < Output Error
        clean_detect -= 1

print(f'Clean Source Detect ratio {clean_detect/len(clean_source)}')
print(len(clean_source))

Clean Source Detect ratio 0.6428571428571429
210


In [7]:
corrupted_perfect_correction = 0
corrupted_semi_perfect_correction = 0
corrupted_source_compile_error_output = 0
corrupted_correct_same_gold = 0
for key in corrupted_source:
    if test_result_dictionary[key][2] == 'CE':
        # Output Compile Error
        corrupted_source_compile_error_output += 1
    elif test_result_dictionary[key][1] > test_result_dictionary[key][2]:
        # Source Error(0.0) < Output Error
        if test_result_dictionary[key][2] == 0.0:
            corrupted_perfect_correction += 1
        else:
            corrupted_semi_perfect_correction += 1

        if int(key.split('_')[1]) in exact_match_index:
            corrupted_correct_same_gold += 1

print(f'Corrupted Source[1.0] Correction Ratio {(corrupted_perfect_correction + corrupted_semi_perfect_correction)/len(corrupted_source)}')
print(f'Corrupted Source[1.0] Prefect Correction Ratio {corrupted_perfect_correction/len(corrupted_source)}')
print(f'Corrupted Source[1.0] Semi Correction Ratio {corrupted_semi_perfect_correction/len(corrupted_source)}')
print(f'Corrupted Source[1.0] Correction (same with Gold) {corrupted_correct_same_gold / (corrupted_perfect_correction + corrupted_semi_perfect_correction)}')

Corrupted Source[1.0] Correction Ratio 0.18181818181818182
Corrupted Source[1.0] Prefect Correction Ratio 0.1
Corrupted Source[1.0] Semi Correction Ratio 0.08181818181818182
Corrupted Source[1.0] Correction (same with Gold) 0.55


In [8]:
minor_issues_perfect_correction = 0
minor_issues_semi_perfect_correction = 0
minor_issues_source_compile_error_output = 0
minor_issues_correct_same_gold = 0
for key in minor_issues_source:
    if test_result_dictionary[key][2] == 'CE':
        # Output Compile Error
        minor_issues_source_compile_error_output += 1
    elif test_result_dictionary[key][1] > test_result_dictionary[key][2]:
        # Source Error(0.0) < Output Error
        if test_result_dictionary[key][2] == 0.0:
            minor_issues_perfect_correction += 1
        else:
            minor_issues_semi_perfect_correction += 1

        if int(key.split('_')[1]) in exact_match_index:
            minor_issues_correct_same_gold += 1

print(f'minor_issues Source(0.0, 1,0) Correction Ratio {round((minor_issues_perfect_correction + minor_issues_semi_perfect_correction)/len(minor_issues_source), 4)}')
print(f'minor_issues Source(0.0, 1,0) Prefect Correction Ratio {round(minor_issues_perfect_correction/len(minor_issues_source),4)}')
print(f'minor_issues Source(0.0, 1,0) Semi Correction Ratio {round(minor_issues_semi_perfect_correction/len(minor_issues_source),4)}')
print(f'minor_issues Source(0.0, 1,0) Correction (same with Gold) {minor_issues_correct_same_gold / (minor_issues_perfect_correction + minor_issues_semi_perfect_correction)}')

minor_issues Source(0.0, 1,0) Correction Ratio 0.1875
minor_issues Source(0.0, 1,0) Prefect Correction Ratio 0.1364
minor_issues Source(0.0, 1,0) Semi Correction Ratio 0.0511
minor_issues Source(0.0, 1,0) Correction (same with Gold) 0.5353535353535354


In [9]:
# Correction Rate of Imprefect Sources
print(f'Source data Correction {round((corrupted_perfect_correction + corrupted_semi_perfect_correction + minor_issues_perfect_correction + minor_issues_semi_perfect_correction) / (len(corrupted_source) + len(minor_issues_source)),4)}')
print(f'Source data Perfect Correction {round((corrupted_perfect_correction + minor_issues_perfect_correction) / (len(corrupted_source) + len(minor_issues_source)),4)}')
print(f'Source data Semi Perfect Correction {round((corrupted_semi_perfect_correction + minor_issues_semi_perfect_correction) / (len(corrupted_source) + len(minor_issues_source)),4)}')

Source data Correction 0.1865
Source data Perfect Correction 0.1301
Source data Semi Perfect Correction 0.0564


In [10]:
print(exact_match_index)

[1, 24, 33, 40, 41, 44, 46, 52, 53, 55, 61, 67, 72, 73, 75, 106, 131, 167, 173, 180, 207, 220, 244, 250, 256, 264, 272, 295, 298, 304, 316, 317, 319, 320, 321, 323, 324, 333, 340, 348, 359, 386, 387, 393, 395, 401, 404, 420, 430, 484, 485, 507, 530, 548, 594, 597, 627, 631, 636, 637, 640, 641, 645, 666, 668, 669, 673, 678, 680, 693, 737, 738, 744, 745, 747, 750, 752, 757, 798, 803, 805, 840, 841, 847, 848, 850, 853, 854, 859]


In [11]:
exact_match_gold_nonzero = 0
exact_match_CE = 0
exact_match_source_zero = 0
exact_match_perfect_fix = 0
for key, value in test_result_dictionary.items():
    if int(key.split('_')[1]) in exact_match_index:
        if (value[0] == 'CE') & (value[2] == 'CE'):
            exact_match_CE += 1
        elif value[1] == 0.0:
            exact_match_source_zero += 1
        elif (value[0] != 0.0) & (value[2] != 0.0):
            exact_match_gold_nonzero += 1
        elif (value[0] == 0.0) & (value[2] == 0.0):
            exact_match_perfect_fix += 1

print(f'Exact Match Gold NoneZero {exact_match_gold_nonzero}')
print(f'Exact Match Source Zero {exact_match_source_zero}')
print(f'Exact Match Compile Error {exact_match_CE}')
print(f'Exact Match Perfect Fix {exact_match_perfect_fix}')
print(f'total {len(exact_match_index)}')

Exact Match Gold NoneZero 9
Exact Match Source Zero 18
Exact Match Compile Error 3
Exact Match Perfect Fix 59
total 89
